In [1]:
"""
Santa 2025 - EXACT COMPETITION TREE SHAPE
Single-cell Kaggle solution with official tree geometry
Copy this entire file into a Kaggle notebook cell and run!
"""

import numpy as np
import pandas as pd
from shapely.geometry import Polygon
from shapely import affinity
from dataclasses import dataclass
from typing import List, Tuple

# ============ OFFICIAL COMPETITION TREE SHAPE ============
def create_competition_tree_shape():
    """
    Official Christmas tree shape from competition starter notebook
    Multi-tiered tree with trunk at bottom
    """
    scale_factor = 1.0
    
    # Tree dimensions (from competition)
    trunk_w = 0.15
    trunk_h = 0.2
    base_w = 0.7
    mid_w = 0.4
    top_w = 0.25
    tip_y = 0.8
    tier_1_y = 0.5
    tier_2_y = 0.25
    base_y = 0.0
    trunk_bottom_y = -trunk_h
    
    # Exact vertices from competition (scaled)
    vertices = np.array([
        # Tip
        [0.0 * scale_factor, tip_y * scale_factor],
        # Right side - Top Tier
        [top_w / 2 * scale_factor, tier_1_y * scale_factor],
        [top_w / 4 * scale_factor, tier_1_y * scale_factor],
        # Right side - Middle Tier
        [mid_w / 2 * scale_factor, tier_2_y * scale_factor],
        [mid_w / 4 * scale_factor, tier_2_y * scale_factor],
        # Right side - Bottom Tier
        [base_w / 2 * scale_factor, base_y * scale_factor],
        # Right Trunk
        [trunk_w / 2 * scale_factor, base_y * scale_factor],
        [trunk_w / 2 * scale_factor, trunk_bottom_y * scale_factor],
        # Left Trunk
        [-trunk_w / 2 * scale_factor, trunk_bottom_y * scale_factor],
        [-trunk_w / 2 * scale_factor, base_y * scale_factor],
        # Left side - Bottom Tier
        [-base_w / 2 * scale_factor, base_y * scale_factor],
        # Left side - Middle Tier
        [-mid_w / 4 * scale_factor, tier_2_y * scale_factor],
        [-mid_w / 2 * scale_factor, tier_2_y * scale_factor],
        # Left side - Top Tier
        [-top_w / 4 * scale_factor, tier_1_y * scale_factor],
        [-top_w / 2 * scale_factor, tier_1_y * scale_factor],
    ])
    
    return vertices

print("Competition tree shape loaded!")
print("Tree dimensions: 0.7 wide × 1.0 tall (tip to trunk bottom)")

# ============ CORE CLASSES ============
@dataclass
class TreeShape:
    vertices: np.ndarray
    def to_polygon(self): return Polygon(self.vertices)
    def get_rotated(self, deg): return affinity.rotate(self.to_polygon(), deg, origin=(0,0))
    def get_transformed(self, x, y, deg): return affinity.translate(self.get_rotated(deg), xoff=x, yoff=y)

@dataclass
class PlacedTree:
    tree_id: int
    x: float
    y: float
    deg: float
    polygon: Polygon

class TreePacker:
    def __init__(self, tree_shape):
        self.tree_shape = tree_shape
        self.placed_trees = []
    
    def check_collision(self, poly):
        return any(poly.intersects(p.polygon) and poly.intersection(p.polygon).area > 1e-6 
                   for p in self.placed_trees)
    
    def place_tree(self, tree_id, x, y, deg):
        poly = self.tree_shape.get_transformed(x, y, deg)
        if self.check_collision(poly): return False
        self.placed_trees.append(PlacedTree(tree_id, x, y, deg, poly))
        return True
    
    def get_box_size(self):
        coords = np.vstack([np.array(p.polygon.exterior.coords) for p in self.placed_trees])
        return max(coords.max(axis=0) - coords.min(axis=0))
    
    def center(self):
        coords = np.vstack([np.array(p.polygon.exterior.coords) for p in self.placed_trees])
        center = (coords.max(axis=0) + coords.min(axis=0)) / 2
        for p in self.placed_trees:
            p.x -= center[0]
            p.y -= center[1]
            p.polygon = self.tree_shape.get_transformed(p.x, p.y, p.deg)
    
    def clear(self):
        self.placed_trees = []

# ============ GRID PACKING ============
def grid_pack(tree_shape, n, angles=[0,30,45,60,90]):
    best_score, best = float('inf'), []
    packer = TreePacker(tree_shape)
    
    for cols in range(1, n+1):
        rows = (n + cols - 1) // cols
        for angle in angles:
            packer.clear()
            bounds = tree_shape.get_rotated(angle).bounds
            sx, sy = (bounds[2]-bounds[0])*1.1, (bounds[3]-bounds[1])*1.1
            
            tid = 0
            for r in range(rows):
                for c in range(cols):
                    if tid >= n: break
                    if not any(packer.place_tree(tid, c*sx, r*sy, angle+off) for off in [0,5,-5]):
                        tid = -1
                        break
                    tid += 1
                if tid < 0: break
            
            if len(packer.placed_trees) == n:
                packer.center()
                score = (packer.get_box_size()**2) / n
                if score < best_score:
                    best_score, best = score, [PlacedTree(p.tree_id,p.x,p.y,p.deg,p.polygon) for p in packer.placed_trees]
    
    return best_score, best

# ============ GENETIC ALGORITHM ============
def genetic_pack(tree_shape, n, pop_size=50, gens=100):
    packer = TreePacker(tree_shape)
    
    def evaluate(genes):
        packer.clear()
        for i in range(n):
            if not packer.place_tree(i, genes[i*3], genes[i*3+1], genes[i*3+2]%360):
                return 1e6, []
        packer.center()
        return (packer.get_box_size()**2)/n, [PlacedTree(p.tree_id,p.x,p.y,p.deg,p.polygon) for p in packer.placed_trees]
    
    pop = [np.concatenate([np.random.randn(n*2)*2, np.random.uniform(0,360,n)]).reshape(-1,3).flatten() for _ in range(pop_size)]
    fits = [evaluate(ind) for ind in pop]
    best_idx = min(range(len(fits)), key=lambda i: fits[i][0])
    best_score, best = fits[best_idx]
    
    for gen in range(gens):
        selected = [pop[min(np.random.choice(len(pop), 5), key=lambda i: fits[i][0])].copy() for _ in range(pop_size)]
        next_pop = []
        for i in range(0, len(selected), 2):
            if i+1 < len(selected):
                pt = np.random.randint(1, len(selected[i]))
                c1, c2 = np.concatenate([selected[i][:pt], selected[i+1][pt:]]), np.concatenate([selected[i+1][:pt], selected[i][pt:]])
                for c in [c1, c2]:
                    for j in range(len(c)):
                        if np.random.random() < 0.1:
                            c[j] = np.random.uniform(0,360) if j%3==2 else c[j]+np.random.normal(0,0.5)
                next_pop.extend([c1, c2])
        
        for idx in sorted(range(len(fits)), key=lambda i: fits[i][0])[:5]:
            next_pop.append(pop[idx].copy())
        
        pop = next_pop[:pop_size]
        fits = [evaluate(ind) for ind in pop]
        idx = min(range(len(fits)), key=lambda i: fits[i][0])
        if fits[idx][0] < best_score:
            best_score, best = fits[idx]
        
        if gen % 20 == 0:
            print(f"    Gen {gen}: best={best_score:.4f}")
    
    return best_score, best

# ============ SOLVER ============
def solve(tree_shape, n):
    print(f"n={n:3d}:", end=" ")
    
    grid_score, grid_place = grid_pack(tree_shape, n)
    results = [('grid', grid_score, grid_place)]
    print(f"grid={grid_score:.4f}", end=" ")
    
    if n <= 30:  # Use GA for small configs
        ga_score, ga_place = genetic_pack(tree_shape, n, pop_size=max(30,n*2), gens=min(100,200//n))
        if ga_place:
            results.append(('ga', ga_score, ga_place))
            print(f"ga={ga_score:.4f}", end=" ")
    
    results.sort(key=lambda x: x[1])
    strategy, score, placement = results[0]
    print(f"→ {strategy} wins")
    return strategy, score, placement

# ============ CREATE TREE ============
tree_vertices = create_competition_tree_shape()
tree = TreeShape(tree_vertices)

# ============ TEST ON SMALL RANGE ============
print("\n" + "="*60)
print("TESTING ON 1-10 TREES (OFFICIAL TREE SHAPE)")
print("="*60)

test_solutions = {}
for n in range(1, 11):
    strategy, score, placement = solve(tree, n)
    test_solutions[n] = (score, placement, strategy)

test_score = sum(s[0] for s in test_solutions.values())
print("="*60)
print(f"Test score (1-10): {test_score:.4f}")
print("="*60)

# Count strategy wins
from collections import Counter
strategies = [s[2] for s in test_solutions.values()]
strategy_counts = Counter(strategies)
print(f"\nStrategy wins: {dict(strategy_counts)}")

# ============ GENERATE TEST SUBMISSION ============
rows = []
for n, (score, placement, _) in test_solutions.items():
    for tree in placement:
        rows.append({
            'id': f"{n:03d}_{tree.tree_id}",
            'x': f"s{tree.x:.10f}",
            'y': f"s{tree.y:.10f}",
            'deg': f"s{tree.deg:.10f}"
        })

df = pd.DataFrame(rows)
df.to_csv('submission_test.csv', index=False)
print(f"\n✓ Test submission saved: {len(rows)} trees")
print("\nFirst 10 rows:")
print(df.head(10))

# ============ UNCOMMENT TO RUN FULL (1-200) ============
# print("\n" + "="*60)
# print("RUNNING FULL COMPETITION (1-200 TREES)")
# print("="*60)
# print("This will take 30-60 minutes...\n")
# 
# full_solutions = {}
# for n in range(1, 201):
#     strategy, score, placement = solve(tree, n)
#     full_solutions[n] = (score, placement, strategy)
#     if n % 10 == 0:
#         partial_score = sum(full_solutions[i][0] for i in range(1, n+1))
#         print(f"Progress: {n}/200 | Cumulative score: {partial_score:.4f}")
# 
# total_score = sum(s[0] for s in full_solutions.values())
# print("="*60)
# print(f"FINAL SCORE: {total_score:.4f}")
# print("="*60)
# 
# # Generate final submission
# rows = []
# for n, (score, placement, _) in full_solutions.items():
#     for tree in placement:
#         rows.append({
#             'id': f"{n:03d}_{tree.tree_id}",
#             'x': f"s{tree.x:.10f}",
#             'y': f"s{tree.y:.10f}",
#             'deg': f"s{tree.deg:.10f}"
#         })
# 
# df = pd.DataFrame(rows)
# df.to_csv('submission.csv', index=False)
# print(f"\n✓ FINAL SUBMISSION: {len(rows)} trees")
# print("Download 'submission.csv' and submit to Kaggle!")

print("\n" + "="*60)
print("READY TO GO!")
print("="*60)
print("Next: Uncomment the 'RUN FULL' section above and run again")

Competition tree shape loaded!
Tree dimensions: 0.7 wide × 1.0 tall (tip to trunk bottom)

TESTING ON 1-10 TREES (OFFICIAL TREE SHAPE)
n=  1: grid=0.6613     Gen 0: best=0.6631
    Gen 20: best=0.6631
    Gen 40: best=0.6631
    Gen 60: best=0.6631
    Gen 80: best=0.6631
ga=0.6631 → grid wins
n=  2: grid=1.0804     Gen 0: best=195.5259
    Gen 20: best=48.6296
    Gen 40: best=1.6821
    Gen 60: best=0.5687
    Gen 80: best=0.5378
ga=0.5328 → ga wins
n=  3: grid=0.9720     Gen 0: best=1878.3915
    Gen 20: best=1513.6869
    Gen 40: best=1232.9221
    Gen 60: best=1003.3920
ga=970.8695 → grid wins
n=  4: grid=0.7290     Gen 0: best=1636.1273
    Gen 20: best=1323.9620
    Gen 40: best=1094.3710
ga=1021.3897 → grid wins
n=  5: grid=1.0035     Gen 0: best=2136.8981
    Gen 20: best=1832.7685
ga=1569.8074 → grid wins
n=  6: grid=0.8363     Gen 0: best=8017.4061
    Gen 20: best=7359.0824
ga=7227.8122 → grid wins
n=  7: grid=0.9673     Gen 0: best=4295.8087
    Gen 20: best=3239.5181
ga=3